In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd

In [2]:
user='G'

if user=='S':
    mandat_dep=pd.read_csv("C:/Users/sylva/Desktop/git/Projet-Python-deputes/liste_mandat.csv")#on récupère le csv de la liste des mandats
    df_depu=pd.read_csv("C:/Users/sylva/Desktop/git/Projet-Python-deputes/listeDepute_complete.csv")

if user=='G':
    mandat_dep=pd.read_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/liste_mandatdef.csv") 
    df_depu=pd.read_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/listeDepute_complete.csv")

<h1> I. Base mandat

In [5]:
debut_mandat=[]
for dates in mandat_dep["Mandat"]:
    dates=str(dates).split()
    debut_mandat.append(' '.join(dates[1:4]))
    
mandat_dep["Debut Mandat"]=np.array(debut_mandat)
mandat_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Nom,Debut Mandat
0,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,Damien Meslot,19 juin 2002
1,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,Damien Meslot,20 juin 2007


In [9]:
df_depu[["Nom","Né le"]].head(2)
mandat_dep[["Nom","Debut Mandat"]].head(2)
age_dep=mandat_dep.set_index('Nom').join(df_depu[["Nom","Né le"]].set_index('Nom'), on='Nom')
mandat_dep.reset_index()
mois={'janvier':1,'février':2, 'mars':3, 'avril':4, 'mai':5, 'juin':6, 'juillet':7, 'août':8, 'septembre':9, 'octobre':10, 'novembre':11, 'décembre':12}
def transtype_date(x):
    x=str(x)        
    date_texte=x.split() 
    if len(date_texte)>1 :
        if  date_texte[1] in (mois.keys()):
            if date_texte[0]=="1er":
                date_texte[0]='1'
            date_texte[1]=str(mois[date_texte[1]])
            date_nombre='/'.join(date_texte)
            return pd.to_datetime(date_nombre)
    else:
        return pd.NaT


age_dep["Date de naissance"]=age_dep["Né le"].apply(lambda x: transtype_date(x))
age_dep["Date debut mandat"]=age_dep["Debut Mandat"].apply(lambda x: transtype_date(x))
age_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Date de naissance,Date debut mandat
Nom,,,,,,,,,,
Damien Meslot,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,19 juin 2002,11 novembre 1964,1964-11-11,2002-06-19
Damien Meslot,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,20 juin 2007,11 novembre 1964,1964-11-11,2007-06-20


In [10]:
age_dep['age depute']=age_dep['Date debut mandat']-age_dep['Date de naissance']#on crée la variable age au debut du mandat, en jour
age_dep['age en année']=age_dep['age depute'].apply(lambda x: float(x.days / 365.25))#et on la transforme en années
age_dep=age_dep[['Département','Groupe', 'Législature', 'Mandat', 'Régime politique', 'age en année','Date debut mandat']]

In [28]:
age_dep.head(2)

,Département,Groupe,Législature,Mandat,Régime politique,age en année,Date debut mandat
Nom,,,,,,,
Damien Meslot,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,37.601643,2002-06-19
Damien Meslot,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,42.603696,2007-06-20


<h1> II.Table député

<h3> Nombre de mandat par député

In [54]:
nb_mandat=age_dep.groupby("Nom").count()["age en année"]
nb_mandat=pd.DataFrame(nb_mandat).rename(columns={"age en année":"nombre de mandat"})
nb_mandat.head()

,nombre de mandat
Nom,
"(Louis, Camille, Alexandre, Marie Dubochet",1
"Aaron, Jean-François Crassous",1
Abbas Gueye,1
Abbes Moulessehoul,1
Abdallah Tebib,1


In [37]:
df_depu_b=df_depu.join(nb_mandat,on="Nom",lsuffix=" ")
df_depu_b.sample(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat
13141,13141,"Louis, Léopold Buquet",5 mai 1768,Vosges,Inconnu,3.0
14168,14168,"Jean-Baptiste, Claude, Henri Dupuy",18 août 1759,Loire,Inconnu,2.0
198,198,Marion Lenne,20 novembre 1974,NaN,NaN,NaN
8708,8708,Félix Vogeli,14 septembre 1832,Nord,Isère,2.0
15065,15065,"Ignace, Joseph, Delfin Haudouart",7 avril 1753,Inconnu,Inconnu,1.0


In [55]:
premier_mandat=age_dep.groupby("Nom")["age en année"].min()
premier_mandat.head()

Nom
(Louis, Camille, Alexandre, Marie Dubochet    45.689254
Aaron, Jean-François Crassous                 49.267625
Abbas Gueye                                   37.470226
Abbes Moulessehoul                            38.811773
Abdallah Tebib                                60.766598
Name: age en année, dtype: float64

In [ ]:
age_dep=age_dep.reset_index()

In [58]:
premier_mandat_dpt=age_dep.groupby("Nom")[["Département","Nom"]].head(1)
premier_mandat_dpt=pd.DataFrame(premier_mandat_dpt).rename(columns={"Département":"premier dpt"}).set_index("Nom")
premier_mandat_dpt.head(3)

,premier dpt
Nom,
Damien Meslot,Territoire-de-Belfort
Christine Cloarec-Le Nabour,Ille-et-Vilaine
Pascal Terrasse,Ardèche


<h3> Caractéristiques du premier mandat

In [ ]:
df_depu_b=df_depu_b.join(premier_mandat,on="Nom",lsuffix="")
df_depu_b=df_depu_b.join(premier_mandat_dpt,on="Nom",lsuffix="")

In [51]:
df_depu_b.sample(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat,age en année,premier dpt
9383,9383,Jean-Louis Lepouzé,20 janvier 1821,Eure,Eure,4.0,51.783710,Eure
13,13,Marion Maréchal-Le Pen,10 décembre 1989,NaN,NaN,NaN,NaN,NaN
5583,5583,Marcel Deschaseaux,16 janvier 1885,Vosges,Vosges,1.0,53.954825,Vosges
7104,7104,"Léon, Hubert Charpentier",22 août 1859,Ardennes,Ardennes,1.0,55.118412,Ardennes
15114,15114,"Jean, Gérard Lacuée de Cessac",4 novembre 1752,Paris,Inconnu,1.0,38.746064,Lot-et-Garonne


In [60]:
df_depu_b["élection locale"]=(" "+df_depu_b["Département de naissance"])==df_depu_b["premier dpt"]

In [62]:
df_depu_b.sample(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat,age en année,premier dpt,élection locale
5456,5456,Jean-Baptiste Saget,6 septembre 1887,Jura,Paris,2.0,32.473648,Territoire-de-Belfort,False
5398,5398,"Henri, Jean-Claude, Eugène Malet",15 juillet 1888,Charente,Paris,1.0,44.054757,Charente,True
13786,13786,"Louis, Urbain Bruë",4 août 1762,Morbihan,Morbihan,2.0,31.638604,Morbihan,True
12905,12905,"Jacques, Noel Rifaut",15 février 1771,Inconnu,Inconnu,1.0,44.717317,Eure-et-Loir,False
1183,1183,Didier Boulaud,4 septembre 1950,Allier,Inconnu,2.0,42.904860,Nièvre,False


<h1> III. Base département

In [14]:
dep_possible=list(mandat_dep['Département'].unique())#une liste de tous les départements
dep_possible.remove(" Département d'élection inconnu")
dep_possible.remove(np.nan)

In [17]:
dep_possible[:3]

[' Territoire-de-Belfort', ' Ille-et-Vilaine', ' Ardèche']

In [115]:
def pourcentageTerroir(basedep) :#une fonction qui donne le pourcentage de mandats occupés par des députés nés dans leur département d'élection
    basedep_1=basedep[basedep['Département']==' '+basedep['Département de naissance']]
    nbreterroir=basedep_1[basedep_1['Département de naissance']!='Inconnu']['Département'].count()
    i=basedep[basedep['Département de naissance']!='Inconnu']['Département'].count()
    if i!=0:
        pourcTerroir=nbreterroir/i
    else:
        return np.nan
    return pourcTerroir

In [ ]:
df=age_dep.reset_index()

In [113]:
dept_naiss=age_dep.set_index('Nom').join(df_depu[["Nom","Né le", "Département de naissance"]].set_index('Nom'), on='Nom')
dept_naiss=dept_naiss.reset_index()
dept_naiss.head()

,Nom,level_0,index,Département,Groupe,Législature,Mandat,Régime politique,age en année,Date debut mandat,Né le,Département de naissance
0,Damien Meslot,0,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,37.601643,2002-06-19,11 novembre 1964,Inconnu
1,Damien Meslot,1,1,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,42.603696,2007-06-20,11 novembre 1964,Inconnu
2,Damien Meslot,2,2,Territoire-de-Belfort,Union pour un mouvement populaire,XIVe législature,Du 20 juin 2012 au 20 juin 2017,Cinquième République - Assemblée nationale,47.605749,2012-06-20,11 novembre 1964,Inconnu
3,Christine Cloarec-Le Nabour,3,3,Ille-et-Vilaine,La République en Marche,XVe législature (Législature en cours),Depuis le 21 juin 2017 (mandat en cours),Cinquième République - Assemblée nationale,NaN,NaT,28 octobre 1964,Inconnu
4,Pascal Terrasse,4,4,Ardèche,Socialiste,XIe législature,Du 1er juin 1997 au 18 juin 2002,Cinquième République - Assemblée nationale,32.197125,1997-01-06,26 octobre 1964,Gard


In [116]:
w,x,y,z,u=[],[],[],[],[]
for dep in dep_possible :
    w+=[df[df['Département']==dep]['age en année'].mean()]#on prend la moyenne d'age dans le département
    x+=[len(df[df['Département']==dep])]#on compte le nombre total de mandat dans le département
    y+=[len(df[df['Département']==dep]['Nom'].unique())]#Puis le nombre de députés différent par dpt
    z+=[x[-1]/y[-1]] #Ceci est le nombre moyen de mandat par député
    u+=[pourcentageTerroir(dept_naiss[dept_naiss["Département"]==dep])]
    
    
carac_dep_bis=np.array([w, x, y, z,u])#on crée un dataframe avec les informations
table_depmt = pd.DataFrame(carac_dep_bis, index=['age moyen des deputes', 'nombre_mandat', 'nombre député','nombre moyen de mandats par député', 'locaux'],columns=dep_possible)
table_depmt.T.sample(10)

,age moyen des deputes,nombre_mandat,nombre député,nombre moyen de mandats par député,locaux
Haute-Garonne,49.974552,484.0,223.0,2.170404,0.537002
Saint-Pierre-et-Miquelon,47.747371,23.0,13.0,1.769231,0.000000
Seine-Inférieure,50.799774,569.0,249.0,2.285141,0.600349
Vienne,49.066262,305.0,133.0,2.293233,0.749141
Mayotte,49.566324,12.0,6.0,2.000000,0.416667
Maine-et-Loire,51.041595,473.0,192.0,2.463542,0.000000
Nièvre,48.829330,289.0,125.0,2.312000,0.533923
Guyane,47.010742,50.0,24.0,2.083333,0.829787
Hérault,50.206129,405.0,183.0,2.213115,0.576355
Seine-St-Denis,51.246000,139.0,53.0,2.622642,0.000000
